## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [1]:
# imports
# 导入所需的PyTorch库
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, models, transforms
import os
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# 检查GPU是否可用并设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# download the data (uncomment if to download the data locally)
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
#!unzip hymenoptera_data.zip

In [2]:
# create data loaders
# 定义图像数据集的目录
data_dir = 'hymenoptera_data'

# custom transformer to flatten the image tensors
class ReshapeTransform:# 定义变换函数
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size)
        return result

# transformations used to standardize and normalize the datasets
data_transforms = {# 定义数据集的变换方式
    'train': transforms.Compose([
        transforms.Resize(224), # 调整图像大小
        transforms.CenterCrop(224),# 对中心区域进行裁剪
        transforms.ToTensor(), # 转换成tensor类型
        ReshapeTransform((-1,)) # 数据压平
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),# 调整图像大小
        transforms.CenterCrop(224),# 对中心区域进行裁剪
        transforms.ToTensor(),# 转换成tensor类型
        ReshapeTransform((-1,)) # 数据压平
    ]),
}

# load the correspoding folders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),# 从文件夹中加载数据集
                                          data_transforms[x])
                  for x in ['train', 'val']}

# load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],# 定义训练数据集的 DataLoader
                                            batch_size=len(image_datasets['train']),
                                            shuffle=True)

# 定义测试/验证数据集的 DataLoader
test_dataset = torch.utils.data.DataLoader(image_datasets['val'],
                                           batch_size=len(image_datasets['val']),
                                           shuffle=True)

In [3]:
# 定义一个简单的线性回归模型
class LR(nn.Module):
    def __init__(self, dim):
        super(LR, self).__init__()
          # 线性层
        self.linear = nn.Linear(dim, 1)
          # 将权重和偏置初始化为0
        nn.init.zeros_(self.linear.weight)
        nn.init.zeros_(self.linear.bias)

    def forward(self, x):
        # 计算模型的输出
        x = self.linear(x)
        # 将输出应用Sigmoid函数以获得概率值
        x = torch.sigmoid(x)
        return x 

In [4]:
# 定义一个预测函数
def predict(yhat, y):
     # 去掉最外层的维度
    yhat = yhat.squeeze()
      # 在第0个维度上添加维度，以使其与y的维度匹配
    y = y.unsqueeze(0)
     # 初始化预测输出
    y_prediction = torch.zeros(y.size()[1])
     # 预测类别
    for i in range(yhat.shape[0]):
        if yhat[i] <= 0.5:
            y_prediction[i] = 0
        else:
            y_prediction[i] = 1
    # 计算预测准确度
    return 100 - torch.mean(torch.abs(y_prediction - y)) * 100

In [5]:
# 确定input的维度
dim = train_dataset.dataset[0][0].shape[0]
# 初始化线性回归模型
lrmodel = LR(dim).to(device)
# 定义损失函数
criterion = nn.BCELoss()
# 定义优化器
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001)
"""
这段代码定义了一个简单的线性回归模型 LR，该模型具有一个线性层（self.linear）和一个sigmoid激活函数，用于将线性结果转换为二进制分类输出。然后，定义了一个预测函数 predict，该函数根据模型的输出值对数据进行分类并计算准确度。

在函数之后，定义了一个变量 dim，它存储了数据集中数据的维度。接着，用 LR(dim) 创建了一个线性回归模型，并将其放到 GPU 中。随后，一个 binary cross entropy 损失函数 BCELoss 被实例化，并通过随机梯度下降 SGD 优化器进行优化。优化器的学习率设置为 0.0001。
"""

In [8]:
# training the model
costs = []# 初始化一个空列表以保存每个 epoch 的损失值
# 迭代 100 次
for ITER in range(100):
    lrmodel.train() # 把模型设为训练模式
     # 从训练集中取出一个 batch 的数据
    x, y = next(iter(train_dataset))
     # 从测试集中取出一个 batch 的数据
    test_x, test_y = next(iter(test_dataset))

     # 将数据传输到 GPU 上，并计算模型的输出
    yhat = lrmodel.forward(x.to(device))
     # 计算二元交叉熵损失函数
    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor).to(device))
    # 计算训练集的准确度
    train_pred = predict(yhat, y)

     # 清空梯度
    optimizer.zero_grad()
    # 反向传播计算梯度
    cost.backward()
    # 更新权重
    optimizer.step()
    
     # 把模型设为评估模式，禁用梯度计算
    lrmodel.eval()
    with torch.no_grad():
        # 将测试集数据传输到 GPU 上，并计算模型的输出
        yhat_test = lrmodel.forward(test_x.to(device))
        # 计算测试集的准确度
        test_pred = predict(yhat_test, test_y)
    # 如果 ITER 能被 10 整除，就把损失保存到 costs 中
    if ITER % 10 == 0:
        costs.append(cost)
    # 如果 ITER 能被 10 整除，就打印当前的损失值和训练集、测试集的准确度
    if ITER % 10 == 0:
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER,cost,train_pred,test_pred))

"""
这段代码使用了线性回归模型进行二元分类，并对其进行了训练和评估。
该段代码通过循环 100 次来训练模型，每次循环都从训练数据集中获取一个batch的数据。然后将这些数据传输到GPU上，并使用模型计算出预测值。
在每个循环中，模型都必须计算损失并应用梯度下降进行优化。优化后，模型通过评估集进行测试，并计算测试结果。
如果循环的次数可以被10整除，则损失存入costs列表中，同时打印出损失和训练准确度以及测试准确度的值。
"""

Cost after iteration 0: 0.6931471228599548 | Train Acc: 50.40983581542969 | Test Acc: 45.75163269042969
Cost after iteration 10: 0.6691471338272095 | Train Acc: 64.3442611694336 | Test Acc: 54.24836730957031
Cost after iteration 20: 0.6513182520866394 | Train Acc: 68.44261932373047 | Test Acc: 54.24836730957031
Cost after iteration 30: 0.6367825269699097 | Train Acc: 68.03278350830078 | Test Acc: 54.24836730957031
Cost after iteration 40: 0.6245337128639221 | Train Acc: 69.67213439941406 | Test Acc: 54.90196228027344
Cost after iteration 50: 0.6139225363731384 | Train Acc: 70.90164184570312 | Test Acc: 56.20914840698242
Cost after iteration 60: 0.6045235395431519 | Train Acc: 72.54098510742188 | Test Acc: 56.86274337768555
Cost after iteration 70: 0.5960512161254883 | Train Acc: 74.18032836914062 | Test Acc: 57.51633834838867
Cost after iteration 80: 0.5883084535598755 | Train Acc: 73.77049255371094 | Test Acc: 57.51633834838867
Cost after iteration 90: 0.5811557769775391 | Train Acc: 

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)